In [ ]:
import argparse
import io
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
import json

In [38]:
def transcribe_gcs(gcs_uri):
    """Asynchronously transcribes the audio file specified by the gcs_uri."""
    from google.cloud import speech

    client = speech.SpeechClient()

    audio = types.RecognitionAudio(uri=gcs_uri)
    config = types.RecognitionConfig(
        encoding=types.RecognitionConfig.AudioEncoding.FLAC,
        sample_rate_hertz=48000,
        language_code="en-US",
        audio_channel_count=2,
    )

    operation = client.long_running_recognize(config=config, audio=audio)

    print("Waiting for operation to complete...")
    response = operation.result(timeout=2000)

    
    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        print(type(result))
        # The first alternative is the most likely one for this portion.
        #print(u"Transcript: {}".format(result.alternatives[0].transcript))
        #print("Confidence: {}".format(result.alternatives[0].confidence))
    #print(type(response))
    return response

In [35]:
#Writing output back to file in Cloud storage
from google.cloud import storage
import json
from string import punctuation
import os


In [39]:
def upload_file_to_gcs(storage_client, src_filepath: str, dst_bucket_name: str, dst_filename: str):
    """Uploads a file to the bucket."""
    
    bucket = storage_client.get_bucket(dst_bucket_name)
    
    blob = bucket.blob(dst_filename)

    blob.upload_from_filename(src_filepath)

    return print(f'{src_filepath} uploaded to {dst_bucket_name}/{dst_filename}.')


def upload_transcript_to_gcs(src_filename: str, processed_transcript: json, storage_client, dst_bucket_name: str):
    """"""
    dst_filename = os.path.join("PROCESSED_" + src_filename)
    
    tmp_filepath = os.path.join("/tmp", dst_filename)
    
    with open(f'{tmp_filepath}', 'w') as f:
        json.dump(processed_transcript, f)
    
            
    print(f'Processed transcript stored in: {tmp_filepath}')
    
    return upload_file_to_gcs(storage_client, tmp_filepath, dst_bucket_name, dst_filename)
    
def process_transcripts(data, context = None):
    
    # Set up parameters
    FILE = data['name']
    SRC_BUCKET_NAME = data['bucket']
    DST_BUCKET_NAME = os.environ.get('DST_BUCKET_NAME')
    
    print(f'Source Bucket: {SRC_BUCKET_NAME}')
    print(f'Source Filename: {FILE}')
    print(f'Destination Bucket: {DST_BUCKET_NAME}')   
    
    print('Starting Process Transcript...')
    
    # Instantiate a Google Cloud Storage client and specify required bucket and file
    storage_client = storage.Client()
    
    # process transcript from speech to text 
    #gcs_uri="gs://meeting-audio-team13/on_mute_flac.flac"
    gcs_uri="gs://meeting-audio-team13/test_rec.flac"
    processed_transcript = transcribe_gcs(gcs_uri)
    

    # Upload data to gcs
    upload_transcript_to_gcs(FILE, processed_transcript, storage_client, DST_BUCKET_NAME)
    
    print('Process Transcript Complete.')

if __name__ == "__main__":
    
    gcs_test_object = {
        "name": "test_rec.json",
        "bucket": "meeting-transcript-team13",
    }
    # In Cloud Function set these as environment variables
    os.environ["DST_BUCKET_NAME"] = "meeting-transcript-processed-team13"
    
    process_transcripts(gcs_test_object)

Source Bucket: meeting-transcript-team13
Source Filename: test_rec.json
Destination Bucket: meeting-transcript-processed-team13
Starting Process Transcript...
Waiting for operation to complete...
<class 'google.cloud.speech_v1.types.SpeechRecognitionResult'>
<class 'google.cloud.speech_v1.types.SpeechRecognitionResult'>
<class 'google.cloud.speech_v1.types.SpeechRecognitionResult'>
<class 'google.cloud.speech_v1.types.SpeechRecognitionResult'>
<class 'google.cloud.speech_v1.types.SpeechRecognitionResult'>
<class 'google.cloud.speech_v1.types.SpeechRecognitionResult'>
<class 'google.cloud.speech_v1.types.SpeechRecognitionResult'>
<class 'google.cloud.speech_v1.types.SpeechRecognitionResult'>
<class 'google.cloud.speech_v1.types.SpeechRecognitionResult'>
<class 'google.cloud.speech_v1.types.SpeechRecognitionResult'>
<class 'google.cloud.speech_v1.types.SpeechRecognitionResult'>
<class 'google.cloud.speech_v1.types.SpeechRecognitionResult'>
<class 'google.cloud.speech_v1.types.SpeechRecog

TypeError: Object of type LongRunningRecognizeResponse is not JSON serializable